In [1]:
import datetime as dt
import math
import os
import random
import time
from functools import partial
from pathlib import Path

import cv2
import albumentations
import albumentations.pytorch

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.models.resnet import resnet152, resnet50
from torchvision.transforms.transforms import GaussianBlur, RandomRotation
import tqdm
from PIL import Image
from pytz import timezone
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.models import resnet18
from torchvision.transforms import Normalize, Resize, ToTensor

from data_preprocessing.data_split import Run_Split
from utils.util import ensure_dir, prepare_device, notification
from model.model import resnet_finetune


def collate_fn(batch):
    return tuple(zip(*batch))


class Mask_Dataset(object):
    def __init__(self, transforms, name, df):
        self.transforms = transforms
        self.name = name
        self.imgs = list(
            sorted(
                os.listdir(
                    f"/opt/ml/image-classification-level1-12/templates/data/train/image_all/{self.name}_image"
                )
            )
        )
        self.df = df


    def __getitem__(self, idx):
        # img_path = Path(self.df["path"][idx])
        img_path = self.df["path"][idx]
        target = self.df["label"][idx]
        
        # img = cv2.imread(img_path)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # if self.transforms is not None:
        #     augmented = self.transforms(image = img)
        #     img = augmented['image']

        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            img = self.transforms(img)
                
        return img, target
        

    def __len__(self):
        return len(self.imgs)


train_path = "/opt/ml/image-classification-level1-12/templates/data/train"
train_label = pd.read_csv(os.path.join(train_path, "train_with_label.csv"))
run_split = Run_Split(os.path.join(train_path, "image_all"))
fold_num = 5
train_list, val_list = run_split.train_val_split(train_label, fold_num)



data_transform = transforms.Compose(
	[
		Resize((512, 384), Image.BILINEAR),
		# GaussianBlur(3, sigma=(0.1, 2)),
		RandomRotation([-8, +8]),
		ToTensor(),
		Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
	]
)



train_dataset = Mask_Dataset(data_transform, f"train{i}", train_list[i])
train_loader = DataLoader(
	train_dataset,
	batch_size=128,
	# 배치마다 어떤 작업을 해주고 싶을 때, 이미지 크기가 서로 맞지 않는 경우 맞춰줄 때 사용
	collate_fn=collate_fn,
	# 마지막 남은 데이터가 배치 사이즈보다 작을 경우 무시
	drop_last=False,
	#  num_workers=2
)

ModuleNotFoundError: No module named 'model.medel'